In [2]:
!pip install sentencepiece transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00


In [4]:
import warnings

import numpy as np
import pandas as pd
import torch

torch.manual_seed(420)
np.random.seed(420)
warnings.filterwarnings('ignore')

In [5]:
train_df = pd.read_csv('../data/interim/dataset.csv')

In [6]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("ai-forever/ruT5-base")
model = T5ForConditionalGeneration.from_pretrained("ai-forever/ruT5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
def get_paraphrases(sentence, prefix="answer: ", n_predictions=5, top_k=120, max_length=256):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    text = prefix + sentence + " </s>"
    encoding = tokenizer.encode_plus(
        text, pad_to_max_length=True, return_tensors="pt"
    )
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding[
        "attention_mask"
    ].to(device)

    model_output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        do_sample=True,
        max_length=max_length,
        top_k=top_k,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=n_predictions,
    )

    outputs = []
    for output in model_output:
        generated_sent = tokenizer.decode(
            output, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        if (
                generated_sent.lower() != sentence.lower()
                and generated_sent not in outputs
        ):
            outputs.append(generated_sent)
    return outputs

In [8]:
paraphrases = get_paraphrases("Что такое")

for sent in paraphrases:
    print(sent)

: post). тоска)) это приятность)...)
вам на в 2014 будет интересно :) :)).
:УСТУЕМ Т! В очередной) ему :))).))
:)А что такое такое!!) backstyle... :)
есть такое... :) Какое-то она ей такое). --* it style...


In [9]:
prefix = "answer:"
source = "question"
target = "answer"
max_input_length = 128
max_target_length = 128
batch_size = 4

def preprocess_function(examples):
    inputs = [prefix + str(ex) for ex in examples[source]]
    targets = [str(ex) for ex in examples[target]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
from datasets import Dataset, DatasetDict

tds = Dataset.from_pandas(train_df)
ds = DatasetDict()
ds['train'] = tds
ds['validation'] = tds
ds = ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

In [11]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name = "ai-forever/ruT5-base"

args = Seq2SeqTrainingArguments(
    f"../models/{model_name}-finetuned-{source}-to-{target}",
    disable_tqdm=True,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=100,
    predict_with_generate=True,
    report_to='tensorboard',
)

In [12]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
# Start training
trainer.train()

In [39]:
paraphrases = get_paraphrases("Как установить PyCharm?")

for sent in paraphrases:
    print(sent)

Charm — это проект, который помогает справиться с задачей. Если файлов таких нет, то нужно создать в PyCharm самостоятельно - любую файловую программу. Например, pyCharm понимает, что такое PyCharm и ему нужно использовать, чтобы создать файл на устройстве. Чтобы понять, где искать файл, достаточно нажать правой кнопкой мыши на название открытого файла или сам файл в боте И далее запустить в боте название открытого файла или сам файл, чтобы открыть файл в папке. Чтобы открыть папку, Вам потребуется создать файл в папке. Там же вы и найдёте сам файл pyCharm, где Вы и найдёте открытый файл
Добрый день, Булат! Вы задали вопрос по задачи про файл в pyCharm. Что происходит при нажатии «PyCharm»?
Здравствуйте, Максим Можете пожалуйста скинуть вопрос по задаче? Есть несколько вариантов. Например, для входных данных выводить числа через пробел Или использовать команду break... Или нажать правой кнопкой мыши на треугольник рядом с «PyCharm»... Здесь потребуется нажать правой кнопкой мыши на тре

In [26]:
model_path = 'fine_tuned_t5_model'
# Save the fine-tuned model
trainer.save_model(model_path)

In [40]:
import shutil
import os

# Define the directory to be zipped
folder_path = 'fine_tuned_t5_model'
zip_path = 'fine_tuned_t5_model.zip'
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

'/content/fine_tuned_t5_model.zip'